In [18]:
import pandas as pd

In [19]:
diabetes_data = pd.read_csv('/workspaces/machine-learning-python-template-ds-2023/Ricardo/Boosting-Algorithms/diabetes.csv')

diabetes_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
impute_cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

for col in impute_cols:
    diabetes_data[col] = diabetes_data[col].replace(0, diabetes_data[col].mean())


In [6]:
from sklearn.model_selection import train_test_split

x = diabetes_data.drop('Outcome', axis=1)
y = diabetes_data['Outcome']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [7]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [8]:
y_pred = model.predict(x_test)
y_pred_in = model.predict(x_train)


In [9]:
from sklearn.metrics import accuracy_score

print('Accuracy score for test data is:', accuracy_score(y_test, y_pred))
print('Accuracy score for train data is:', accuracy_score(y_train, y_pred_in))

Accuracy score for test data is: 0.7272727272727273
Accuracy score for train data is: 1.0


In [14]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [100, 200, 300], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7]}

xgb1 = XGBClassifier()

grid_search = GridSearchCV(estimator=xgb1, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)

grid_search.fit(x_train, y_train)

best_params = grid_search.best_params_

best_xgb = XGBClassifier(**best_params)
best_xgb.fit(x_train, y_train)

y_pred_best = best_xgb.predict(x_test)
y_pred_in_best = best_xgb.predict(x_train)

accuracy_test = accuracy_score(y_test, y_pred_best)
accuracy_train = accuracy_score(y_train, y_pred_in_best)

print('Accuracy score for test data is:', accuracy_test)
print('Accuracy score for train data is:', accuracy_train)
print('Best parameters are:', best_params)



Fitting 5 folds for each of 27 candidates, totalling 135 fits
Accuracy score for test data is: 0.7445887445887446
Accuracy score for train data is: 0.8640595903165735
Best parameters are: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}


In [15]:
model3 = XGBClassifier(n_estimators=100, learning_rate=0.05, max_depth=3)
model3.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [16]:
y_pred3 = model3.predict(x_test)
y_pred_in3 = model3.predict(x_train)

print('Accuracy score for test data is:', accuracy_score(y_test, y_pred3))
print('Accuracy score for train data is:', accuracy_score(y_train, y_pred_in3))

Accuracy score for test data is: 0.7445887445887446
Accuracy score for train data is: 0.8640595903165735


In [17]:
import xgboost as xgb

In [20]:
dtrain = xgb.DMatrix(x_train, label=y_train)

cv_params = {'max_depth': 3,
    'eta': 0.1,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss'
}

cv_results = xgb.cv(dtrain=dtrain, params=cv_params, nfold=5, num_boost_round=100, early_stopping_rounds=10, as_pandas=True, seed=42)

print(cv_results)

    train-logloss-mean  train-logloss-std  test-logloss-mean  test-logloss-std
0             0.612409           0.002802           0.618052          0.010638
1             0.582724           0.003079           0.593343          0.007640
2             0.558428           0.003918           0.573447          0.006518
3             0.536830           0.004182           0.557500          0.005854
4             0.518286           0.004222           0.543596          0.007783
5             0.500967           0.004989           0.531479          0.009515
6             0.486415           0.005542           0.519897          0.009611
7             0.473299           0.005962           0.510949          0.011072
8             0.461350           0.006795           0.502971          0.011844
9             0.450010           0.006466           0.494940          0.013095
10            0.440753           0.006708           0.489833          0.012943
11            0.431848           0.007473           